In [1]:
%%capture
!pip install transformers torch sentencepiece accelerate bitsandbytes

In [2]:
%%capture
!pip install sacrebleu

In [3]:
from transformers import AutoModelForCausalLM, LlamaTokenizer, pipeline

model_id = "Trendyol/Trendyol-LLM-7b-chat-v0.1"
tokenizer = LlamaTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map='auto',
                                             load_in_8bit=True)

sampling_params = dict(do_sample=True, temperature=0.3, top_k=50, top_p=0.9)

pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                device_map="auto",
                max_new_tokens=1024,
                return_full_text=True,
                repetition_penalty=1.1
               )

DEFAULT_SYSTEM_PROMPT = "Sen yardımcı bir asistansın ve sana verilen talimatlar doğrultusunda en iyi cevabı üretmeye çalışacaksın.\n"

TEMPLATE = (
    "[INST] <<SYS>>\n"
    "{system_prompt}\n"
    "<</SYS>>\n\n"
    "{instruction} [/INST]"
)

def generate_prompt(instruction, system_prompt=DEFAULT_SYSTEM_PROMPT):
    return TEMPLATE.format_map({'instruction': instruction,'system_prompt': system_prompt})

def generate_output(user_query, sys_prompt=DEFAULT_SYSTEM_PROMPT):
    prompt = generate_prompt(user_query, sys_prompt)
    outputs = pipe(prompt,
               **sampling_params
              )
    return outputs[0]["generated_text"].split("[/INST]")[-1]

user_query = "Türkiye'de kaç il var?"
response = generate_output(user_query)
print(response)


/home/adam/miniconda3/envs/myenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 6/6 [00:08<00:00,  1.42s/it]


  Türkiye'de 81 il bulunmaktadır.


In [4]:
user_query = "Şu cümleyi ingilizceye çevir: Nasılsın bugün?"
response = generate_output(user_query)
print(response)

  "How are you today?"


In [5]:
DEFAULT_SYSTEM_PROMPT = "Sen yardımcı bir asistansın ve sana verilen talimatlar doğrultusunda en iyi cevabı üretmeye çalışacaksın.\n"

TEMPLATE = (
    "[INST] <<SYS>>\n"
    "{system_prompt}\n"
    "<</SYS>>\n\n"
    "{instruction} [/INST]"
)

def generate_prompt(instruction, system_prompt=DEFAULT_SYSTEM_PROMPT):
    return TEMPLATE.format_map({'instruction': instruction,'system_prompt': system_prompt})

def generate_output(user_query, sys_prompt=DEFAULT_SYSTEM_PROMPT):
    prompt = generate_prompt(user_query, sys_prompt)
    outputs = pipe(prompt,
               **sampling_params
              )
    return outputs[0]["generated_text"].split("[/INST]")[-1]

In [8]:
instructions = """Sen yardımcı bir asistansın ve sana verilen talimatlar doğrultusunda en iyi cevabı üretmeye çalışacaksın. Görevin sana verilen cümleleri ingilizce diline tercüme etmek. Birkaçta tane örnek:
1. Bunlar bizim kitaplarımız.	-	These are our books.
2. Bunlar benim kalemlerim.	-	These are my pencils.
3.	Ateşin var mı?	-	Do you have a match?
4.	Sam Tom'dan iki yaş küçük.	-	Sam is two years younger than Tom.
5.	Lütfen burayı imzalayın.	-	Please sign your name here.
Bundan sonra sana verilen her cümleyi ingilizce diline tercüme et:"""

user_query = "Lütfen burayı imzalayın."

response = generate_output(f"{instructions} {user_query}")
print(response)


  Please sign your name here.


In [48]:
import sacrebleu

# Example reference and hypothesis translations
references = ["This is a correct translation."]
hypotheses = ["This is a translated text."]

bleu = sacrebleu.corpus_bleu(hypotheses, [references])
print(f"BLEU score: {bleu.score}")

BLEU score: 32.46679154750991


In [15]:
import pandas as pd

df = pd.read_csv("tr-en-tatoeba.tsv", sep='\t', header=None, names=["ID1", "Turkish", "ID2", "English"])
df = df[["Turkish", "English"]]
df.head()

,Turkish,English
0,Devenin belini kıran son saman çöpüdür.,The last straw breaks the camel's back.
1,Bilmiyorum.,I don't know.
2,Mum kendiliğinden söndü.,The candle went out by itself.
3,"On, on bir, on iki, on üç, on dört, on beş, on...","Ten, eleven, twelve, thirteen, fourteen, fifte..."
4,Ek olarak yaşlılar birbirleriyle sosyalleşebil...,In addition many groups have been formed so th...


In [45]:
def llm_func(sentence):
  instructions = """Sen yardımcı bir asistansın ve sana verilen talimatlar doğrultusunda en iyi cevabı üretmeye çalışacaksın. Görevin sana verilen cümleleri ingilizce diline tercüme etmek. Birkaçta tane örnek:
  1. Bunlar bizim kitaplarımız.	-	These are our books.
  2. Bunlar benim kalemlerim.	-	These are my pencils.
  3.	Ateşin var mı?	-	Do you have a match?
  4.	Sam Tom'dan iki yaş küçük.	-	Sam is two years younger than Tom.
  5.	Lütfen burayı imzalayın.	-	Please sign your name here.
  Bundan sonra sana verilen her cümleyi ingilizce diline tercüme et:"""

  user_query = "Lütfen burayı imzalayın."

  response = generate_output(f"{instructions} {sentence}")
  return response

def batch_translate(data):
  results = []
  for i, row in data.iterrows():
    original = row["Turkish"]
    target = row["English"]
    pred = llm_func(original)
    results.append([target, pred])
  return results 

def calculate_bleu(translations):
  total_bleu = 0
  for ref, hyp in translations:
    score = sacrebleu.corpus_bleu([hyp], [[ref]]).score
    total_bleu += score
    print(f"{ref} =? {hyp} -> {round(score, 2)}")
  return total_bleu/len(translations)

In [43]:
%%time
translations = batch_translate(df[:30])

CPU times: user 19.6 s, sys: 0 ns, total: 19.6 s
Wall time: 19.7 s


In [47]:
calculate_bleu(translations)

The last straw breaks the camel's back. =?    The last saman stub is the one that broke the donkey's tail. -> 7.35
I don't know. =?   I don't know. -> 100.0
The candle went out by itself. =?   The candle blew itself out. -> 17.28
Ten, eleven, twelve, thirteen, fourteen, fifteen, sixteen, seventeen, eighteen, nineteen, twenty. =?   On, on bir, on iki, on üç, on dört, on beş, on altı, on yedi, on sekiz, on dokuz, yirmi. -> 2.18
In addition many groups have been formed so that the elderly can socialize with one another and remain active participants in American life. =?   Ayrıca, yaşlılar birbirleriyle sosyalleşebilir ve Amerikan hayatının aktif üyeleri olarak kalabilirler için birçok topluluk oluşturulmuştur. -> 1.52
Some read books just to pass time. =?   Some people read books just to pass the time. -> 46.71
He had dark brown hair. =?   He had brown hair. -> 40.94
This ship is not fit for an ocean voyage. =?   This ship is not suitable for an ocean voyage. -> 65.8
It's this book. =?   

37.50624681906648